In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import folium
from folium import GeoJson
from folium import Choropleth
from folium.plugins import HeatMap
import numpy as np
import matplotlib.pyplot as plt
import json
from datetime import timedelta
import requests
import os
from dotenv import load_dotenv
from geopy.distance import distance

In [ ]:
bus_routes = pd.read_csv("data/bus_routes_full.csv")
bus_stops = pd.read_csv("data/bus_stops_full.csv")
bus_services = pd.read_csv("data/bus_services_full.csv")
passenger_volume_OD = pd.read_csv("data/passenger_volume_OD.csv")
passenger_volume = pd.read_csv("data/passenger_volume.csv")
planning_areas = gpd.read_file("data/MasterPlan2019PlanningAreaBoundaryNoSea.geojson")
mrt_stations = pd.read_csv("data/mrt_stations.csv")


In [ ]:
load_dotenv()
onemap_api_key = os.getenv('OneMap_api_key')

In [ ]:
bus_routes_combined = pd.merge(bus_routes, bus_stops, on='BusStopCode', how='left')

bus_services_subset = bus_services[['ServiceNo', 'Category']].drop_duplicates(subset='ServiceNo')

bus_routes_combined = pd.merge(
    bus_routes_combined,
    bus_services_subset,
    on="ServiceNo",
    how="left"
)

bus_routes_combined.head()



In [ ]:
bus_routes_combined['geometry'] = bus_routes_combined.apply(lambda x: Point((x.Longitude, x.Latitude)), axis=1)

In [ ]:
bus_routes_combined = gpd.GeoDataFrame(bus_routes_combined, geometry='geometry')

bus_routes_combined = bus_routes_combined.set_crs(epsg=4326)

print(bus_routes_combined.head())

  ServiceNo Operator  Direction  StopSequence  BusStopCode  Distance  \
0        10     SBST          1             1        75009       0.0   
1        10     SBST          1             2        76059       0.6   
2        10     SBST          1             3        76069       1.1   
3        10     SBST          1             4        96289       2.3   
4        10     SBST          1             5        96109       2.7   

  WD_FirstBus WD_LastBus SAT_FirstBus SAT_LastBus SUN_FirstBus SUN_LastBus  \
0        0500       2300         0500        2300         0500        2300   
1        0502       2302         0502        2302         0502        2302   
2        0504       2304         0504        2304         0503        2304   
3        0508       2308         0508        2309         0507        2308   
4        0509       2310         0509        2311         0508        2309   

          RoadName           Description  Latitude   Longitude  \
0  Tampines Ctrl 1          Tamp

In [ ]:
bus_routes_combined.to_csv("data/bus_routes_combined.csv", index=False)

In [ ]:
# Prepare LineStrings for all services
routes = (
    bus_routes_combined.sort_values(by=['ServiceNo', 'Direction', 'StopSequence'])
    .groupby(['ServiceNo', 'Direction', 'Category'])['geometry']
    .apply(lambda x: LineString(x.tolist()))
    .reset_index()
)

bus_routes_ls = gpd.GeoDataFrame(routes, geometry='geometry', crs="EPSG:4326")

# Filter for trunk services only to create trunk_routes_ls
trunk_routes_ls = bus_routes_ls[bus_routes_ls['Category'] == 'TRUNK'].copy()

In [ ]:
print(bus_routes_ls.head())
bus_routes_ls.to_file("data/line_string.geojson", driver='GeoJSON')

  ServiceNo  Direction                                           geometry
0        10          1  LINESTRING (103.94339 1.35408, 103.94165 1.352...
1        10          2  LINESTRING (103.76988 1.29425, 103.76908 1.292...
2       100          1  LINESTRING (103.87169 1.35047, 103.87205 1.346...
3       100          2  LINESTRING (103.78932 1.31107, 103.78969 1.309...
4      100A          1  LINESTRING (103.87169 1.35047, 103.87205 1.346...


In [ ]:

def duplicate_entries(df):

    new_rows = []

    for _, row in df.iterrows():
        # Check if the STN_NO contains a '/'
        if '/' in row['STN_NO']:
            # Split the STN_NO
            lines = row['STN_NO'].split('/')
            for line in lines:
                new_row = row.copy()
                new_row['STN_NO'] = line.strip()
                new_rows.append(new_row)
        else:
            new_rows.append(row)

    return pd.DataFrame(new_rows)


mrt_stations_expanded = duplicate_entries(mrt_stations)

mrt_stations_expanded['LINE'] = mrt_stations_expanded['STN_NO'].str.extract(r'([A-Z]+)')

mrt_stations_expanded['NO'] = mrt_stations_expanded['STN_NO'].str.extract(r'(\d+)$')

mrt_stations_expanded['NO'] = mrt_stations_expanded['NO'].astype(int)

mrt_stations_clean = (mrt_stations_expanded.sort_values(['LINE', 'NO']).reset_index(drop=True))

print(mrt_stations_clean.head())

In [ ]:

mrt_gdf = gpd.GeoDataFrame(
   mrt_stations_clean,
    geometry=gpd.points_from_xy(mrt_stations_clean.Longitude, mrt_stations_clean.Latitude),
    crs="EPSG:4326"
)

Creating MRT buffers

In [ ]:
def create_mrt_buffer(mrt_gdf, mrt_line, buffer_distance):

    filtered_gdf = mrt_gdf[mrt_gdf["LINE"] == mrt_line]

    if filtered_gdf.empty:
        raise ValueError(f"No data found for MRT line: {mrt_line}")

    route_line = LineString(filtered_gdf['geometry'].tolist())

    route_ls = gpd.GeoDataFrame({'Line': ['MRT'], 'geometry': [route_line]}, crs="EPSG:4326")

    route_ls = route_ls.to_crs(epsg=32648)

    mrt_buffer = route_ls.buffer(buffer_distance).union_all()

    return mrt_buffer

In [ ]:
DTL_buffer = create_mrt_buffer(mrt_gdf, "DT", 400)
TEL_buffer = create_mrt_buffer(mrt_gdf, "TE", 400)

Calculating bus routes overlap with MRT buffers

In [ ]:
def calculate_bus_overlap(bus_routes_ls, buffer):

    bus_routes_ls = bus_routes_ls.to_crs(epsg=32648)

    def calculate_overlap(route):
        intersection = route.intersection(buffer)
        overlap_length = intersection.length
        route_length = route.length
        overlap_percentage = (overlap_length / route_length) * 100 if route_length > 0 else 0
        return pd.Series({'Overlap Length': overlap_length, 'Overlap Percentage': overlap_percentage})

    buffer_overlap = bus_routes_ls.copy()

    # Apply the overlap calculation
    buffer_overlap[['Overlap Length', 'Overlap Percentage']] = buffer_overlap['geometry'].apply(calculate_overlap)

    # Group by ServiceNo and Direction and aggregate the overlap data
    service_overlap = buffer_overlap.groupby(['ServiceNo', 'Direction'])[['Overlap Length', 'Overlap Percentage']].sum().reset_index()

    return service_overlap

In [ ]:
TEL_bus_overlap = calculate_bus_overlap(trunk_routes_ls, TEL_buffer)
DTL_bus_overlap = calculate_bus_overlap(trunk_routes_ls, DTL_buffer)

combined_bus_overlap = pd.concat([TEL_bus_overlap, DTL_bus_overlap], ignore_index=True)

Find the top 10 bus routes with the most overlap with TEL/DTL

In [ ]:
combined_bus_overlap_sorted = combined_bus_overlap.sort_values(by='Overlap Length', ascending=False)

combined_bus_overlap_sorted = combined_bus_overlap_sorted[combined_bus_overlap_sorted['ServiceNo'].str.match(r'^\d+$')]

combined_bus_overlap_sorted = combined_bus_overlap_sorted[combined_bus_overlap_sorted['Direction'] == 1]

top_10_bus_services = combined_bus_overlap_sorted.drop_duplicates(subset=['ServiceNo']).head(10)

print(top_10_bus_services)

In [ ]:
filtered_bus_routes = top_10_bus_services.merge(bus_routes_combined, on=['ServiceNo', 'Direction'], how='left')

Find the average tap volume to estimate passenger demand for each bus service

In [ ]:

def calculate_average_tap_volume(bus_routes, passenger_volume):

    # Aggregate total tap volumes by PT_CODE
    tap_volume = passenger_volume.groupby('PT_CODE').agg(
        TOTAL_TAP_IN_VOLUME=('TOTAL_TAP_IN_VOLUME', 'sum'),
        TOTAL_TAP_OUT_VOLUME=('TOTAL_TAP_OUT_VOLUME', 'sum')
    ).reset_index()

    bus_stops_passenger_demand = bus_routes.merge(
        tap_volume, left_on='BusStopCode', right_on='PT_CODE', how='left'
    ).drop(columns=['PT_CODE'])

    average_service_volume = (
        bus_stops_passenger_demand
        .groupby('ServiceNo')
        .agg(
            TOTAL_TAP_IN_VOLUME=('TOTAL_TAP_IN_VOLUME', 'sum'),
            TOTAL_TAP_OUT_VOLUME=('TOTAL_TAP_OUT_VOLUME', 'sum'),
            NumBusStops=('BusStopCode', 'nunique')
        )
        .reset_index()
    )

    # Calculate average tap-in and tap-out volumes and combined average tap volume
    average_service_volume['AVG_TAP_VOLUME'] = (
        (average_service_volume['TOTAL_TAP_IN_VOLUME'] + average_service_volume['TOTAL_TAP_OUT_VOLUME'])
        / average_service_volume['NumBusStops']
    )

    average_service_volume = average_service_volume[['ServiceNo', 'AVG_TAP_VOLUME']]

    return average_service_volume

passenger_volume_sept = passenger_volume[
    (passenger_volume['YEAR_MONTH'] == "2024-09")]

avg_tap_volume = calculate_average_tap_volume(filtered_bus_routes, passenger_volume_sept)
print(avg_tap_volume.sort_values(by='AVG_TAP_VOLUME', ascending=False))

top10_avg_tap_volume=avg_tap_volume

In [ ]:
def get_passenger_demand(bus_routes_combined, passenger_volume, year_month):

    passenger_volume= passenger_volume[(passenger_volume['YEAR_MONTH'] == year_month)]
    # Group by PT_CODE and sum the relevant passenger volumes
    passenger_volume = passenger_volume.groupby('PT_CODE').agg({
        'TOTAL_TAP_IN_VOLUME': 'sum',
        'TOTAL_TAP_OUT_VOLUME': 'sum'
    }).reset_index()

    # Count the number of services for each bus stop
    service_counts = bus_routes_combined.groupby('BusStopCode')['ServiceNo'].nunique().reset_index()
    service_counts.columns = ['BusStopCode', 'ServiceCount']

    avg_tap_volume = pd.merge(service_counts, passenger_volume, left_on="BusStopCode", right_on='PT_CODE', how='left')

    # Clean up duplicate PT_CODE columns (from both merges)
    avg_tap_volume.drop(columns=['PT_CODE'], inplace=True)

    avg_tap_volume['avg_tap_in'] = avg_tap_volume['TOTAL_TAP_IN_VOLUME'] / avg_tap_volume['ServiceCount'].fillna(0).round().astype(int)
    avg_tap_volume['avg_tap_out'] = avg_tap_volume['TOTAL_TAP_OUT_VOLUME'] / avg_tap_volume['ServiceCount'].fillna(0).round().astype(int)

    bus_stops_passenger_demand = bus_routes_combined.merge(
        avg_tap_volume, on='BusStopCode', how='left'
    )

    return bus_stops_passenger_demand


In [ ]:
def calculate_average_tap_volume(bus_stops_passenger_demand):

    average_service_volume = (
        bus_stops_passenger_demand
        .groupby('ServiceNo')
        .agg(
            TOTAL_TAP_IN_VOLUME=('avg_tap_in', 'sum'),
            TOTAL_TAP_OUT_VOLUME=('avg_tap_out', 'sum'),
            NumBusStops=('BusStopCode', 'nunique')
        )
        .reset_index()
    )

    # Calculate average tap-in and tap-out volumes and combined average tap volume
    average_service_volume['AVG_TAP_VOLUME'] = (
        (average_service_volume['TOTAL_TAP_IN_VOLUME'] + average_service_volume['TOTAL_TAP_OUT_VOLUME'])
        / average_service_volume['NumBusStops']
    )

    average_service_volume = average_service_volume[['ServiceNo', 'AVG_TAP_VOLUME']]

    return average_service_volume

In [ ]:
avg_tap_volume_busstops = get_passenger_demand(bus_routes_combined, passenger_volume, "2024-09")
top10_avg_tap_volume_busstops = avg_tap_volume_busstops[
    avg_tap_volume_busstops[['ServiceNo', 'Direction']].apply(tuple, 1).isin(
        filtered_bus_routes[['ServiceNo', 'Direction']].apply(tuple, 1)
    )
]

top10_avg_tap_volume = calculate_average_tap_volume(top10_avg_tap_volume_busstops)
print(top10_avg_tap_volume.sort_values(by='AVG_TAP_VOLUME', ascending=False))

Find how many bus stops in a route are within 400m of an MRT station


In [ ]:
def bus_stops_near_mrt(bus_routes_combined, mrt_gdf, buffer_distance=400):
    results = []

    # Set CRS for MRT stations
    mrt_gdf = mrt_gdf.to_crs(epsg=32648)

    # Create a buffer around each MRT station
    mrt_buffers = mrt_gdf.copy()
    mrt_buffers['geometry'] = mrt_buffers.geometry.buffer(buffer_distance)

    # Iterate over each unique bus route and direction
    unique_routes_directions = bus_routes_combined[['ServiceNo', 'Direction']].drop_duplicates()

    for _, row in unique_routes_directions.iterrows():
        route = row['ServiceNo']
        direction = row['Direction']

        # Filter bus stops for the current route and direction
        bus_stops = bus_routes_combined[
            (bus_routes_combined['ServiceNo'] == route) &
            (bus_routes_combined['Direction'] == direction)
        ]

        # Create a GeoDataFrame for bus stops
        bus_stops_gdf = gpd.GeoDataFrame(bus_stops, geometry=bus_stops['geometry'], crs="EPSG:4326")

        bus_stops_gdf = bus_stops_gdf.to_crs(epsg=32648)

        # Check for bus stops within the MRT buffers
        bus_stops_gdf['within_mrt_buffer'] = bus_stops_gdf.geometry.apply(
            lambda bus_stop: any(mrt_buffer.intersects(bus_stop) for mrt_buffer in mrt_buffers.geometry)
        )

        # Count the total number of bus stops and those within the buffer
        total_bus_stops = len(bus_stops_gdf)
        count_within_buffer = bus_stops_gdf['within_mrt_buffer'].sum()

        # Calculate the percentage of bus stops near MRT
        percentage_within_buffer = (count_within_buffer / total_bus_stops * 100) if total_bus_stops > 0 else 0

        # Store the results
        results.append({
            'Route': route,
            'Direction': direction,
            'Total Bus Stops': total_bus_stops,
            'Bus Stops Within Buffer': count_within_buffer,
            'Percentage Within Buffer': percentage_within_buffer
        })

    return results


In [ ]:
results = bus_stops_near_mrt(filtered_bus_routes, mrt_gdf, buffer_distance=400)

results_df = pd.DataFrame(results)


results_df = results_df.rename(columns={
    "Route": "ServiceNo",
    "Percentage Within Buffer": "Percentage Near MRT"
})

top10_near_mrt = results_df[['ServiceNo', 'Percentage Near MRT']]

To find how many alternative bus services overlap with at least 30% of each shortlisted route.

In [ ]:
def count_services_with_overlap(shortlisted_services, bus_routes_ls, threshold):

    overlap_results = []

    for _, shortlisted_row in shortlisted_services.iterrows():
        service_no = shortlisted_row['ServiceNo']
        shortlisted_geom = bus_routes_ls.loc[bus_routes_ls['ServiceNo'] == service_no, 'geometry'].values[0]

        shortlisted_length = shortlisted_geom.length
        overlap_count = 0
        for _, existing_route in bus_routes_ls.iterrows():
            existing_service_no = existing_route['ServiceNo']
            existing_geom = existing_route['geometry']
            existing_route_length = existing_geom.length

            if service_no == existing_service_no:
                continue

            intersection = shortlisted_geom.intersection(existing_geom)
            intersection_length = intersection.length if intersection.is_valid else 0

            overlap_percentage = (intersection_length / shortlisted_length) * 100 if shortlisted_length > 0 else 0

            if overlap_percentage >= threshold:
                overlap_count += 1

        overlap_results.append({
            'ServiceNo': service_no,
            'Overlap Count': overlap_count
        })

    overlap_count_df = pd.DataFrame(overlap_results)
    return overlap_count_df

In [ ]:
top10_alt_buses = count_services_with_overlap(top_10_bus_services, bus_routes_ls, threshold=30)
print(top10_alt_buses)

Combining metrics for the 10 shortlisted bus routes

In [ ]:
top_10_metrics = pd.merge(top_10_bus_services, top10_avg_tap_volume, on="ServiceNo", how="left")
top_10_metrics = pd.merge(top_10_metrics, top10_near_mrt, on = 'ServiceNo', how = "left")
top_10_metrics = pd.merge(top_10_metrics, top10_alt_buses, on = 'ServiceNo', how = "left")

print(top_10_metrics)

Normalising metrics and creating a scoring framework

In [ ]:
from sklearn.preprocessing import MinMaxScaler

top_10_metrics_rank = top_10_metrics

columns_to_normalize = ['Overlap Length', 'Overlap Percentage', 'AVG_TAP_VOLUME', 'Percentage Near MRT', 'Overlap Count']


# Apply min-max normalization
scaler = MinMaxScaler()
top_10_metrics_rank[columns_to_normalize] = scaler.fit_transform(top_10_metrics_rank[columns_to_normalize])

top_10_metrics_rank['AVG_TAP_VOLUME'] = 1 - top_10_metrics_rank['AVG_TAP_VOLUME']

# Inverted scaling for AVG_TAP_VOLUME so that higher demand means a lower score

# Define weights for each metric
weights = {
    'Overlap Length': 0,
    'Overlap Percentage': 5,
    'AVG_TAP_VOLUME': 2,
    'Percentage Near MRT': 1,
    'Overlap Count': 2
}

top_10_metrics_rank['Final Score'] = (
    top_10_metrics_rank['Overlap Length'] * weights['Overlap Length'] +
    top_10_metrics_rank['Overlap Percentage'] * weights['Overlap Percentage'] +
    top_10_metrics_rank['AVG_TAP_VOLUME'] * weights['AVG_TAP_VOLUME'] +
    top_10_metrics_rank['Percentage Near MRT'] * weights['Percentage Near MRT'] +
    top_10_metrics_rank['Overlap Count'] * weights['Overlap Count']
)


top_10_metrics_rank = top_10_metrics_rank.sort_values(by='Final Score', ascending=False)

print(top_10_metrics_rank[['ServiceNo', 'Final Score'] + columns_to_normalize])


Calculating the passenger demand for each bus stop along the bus routes


In [ ]:
def filter_avg_tap_volume(avg_tap_volume, ServiceNo):
    filtered_avg_tap_volume = avg_tap_volume[avg_tap_volume['ServiceNo'] == str(ServiceNo)]

    filtered_avg_tap_volume['avg_tap_in'].iloc[-1] = 0
    filtered_avg_tap_volume['avg_tap_out'].iloc[0] = 0

    # Sort the DataFrame by StopSequence for ordered plotting
    filtered_avg_tap_volume = filtered_avg_tap_volume.sort_values(by='StopSequence')

    # Convert BusStopCode to a categorical variable ordered by StopSequence
    filtered_avg_tap_volume['BusStopCode'] = pd.Categorical(filtered_avg_tap_volume['BusStopCode'],
                                                       categories=filtered_avg_tap_volume['BusStopCode'].unique(),
                                                       ordered=True)
    return filtered_avg_tap_volume


Plotting the passenger demand for each bus route

In [ ]:
passenger_volume_am = passenger_volume[
        (passenger_volume['TIME_PER_HOUR'].isin([8, 9])) &
        (passenger_volume["DAY_TYPE"] == "WEEKDAY")]

def plot_monthly_passenger_demand(bus_routes_combined, passenger_volume, service_no, months=["2024-07", "2024-08", "2024-09"]):

    passenger_demand_data = {}
    for month in months:
        passenger_demand = get_passenger_demand(bus_routes_combined, passenger_volume, month)

        # Merge with bus routes to filter for specified service number and direction
        passenger_demand = passenger_demand[
            (passenger_demand['ServiceNo'] == service_no) & (passenger_demand['Direction'] == 1)
        ].sort_values(by='StopSequence')

        # Set tap-in/out values to 0 for first and last stops
        if len(passenger_demand) > 0:
            passenger_demand['avg_tap_in'].iloc[-1] = 0
            passenger_demand['avg_tap_out'].iloc[0] = 0

        # Set BusStopCode as a categorical variable for ordered plotting
        passenger_demand['BusStopCode'] = pd.Categorical(
            passenger_demand['BusStopCode'],
            categories=passenger_demand['BusStopCode'].unique(),
            ordered=True
        )

        passenger_demand_data[month] = passenger_demand

    # Create side-by-side subplots
    fig, axs = plt.subplots(1, 2, figsize=(20, 6))

    # Plot average tap-in and tap-out volumes for each month
    for month, color in zip(months, ['blue', 'orange', 'green']):
        axs[0].plot(
            range(len(passenger_demand_data[month])),
            passenger_demand_data[month]['avg_tap_in'],
            marker='o',
            color=color,
            linestyle='-',
            label=month
        )
        axs[1].plot(
            range(len(passenger_demand_data[month])),
            passenger_demand_data[month]['avg_tap_out'],
            marker='o',
            color=color,
            linestyle='-',
            label=month
        )

    # Configure tap-in subplot
    axs[0].set_xlabel('Bus Stop')
    axs[0].set_ylabel('Average Tap-In Volume')
    axs[0].set_title(f'Average Tap-In Volume for Bus Service {service_no} by Stop Sequence')
    axs[0].set_xticks(range(len(passenger_demand_data["2024-07"])))
    axs[0].set_xticklabels(passenger_demand_data[months[0]]['Description'], rotation=90, ha='right')
    axs[0].grid(axis='y')
    axs[0].legend()

    # Configure tap-out subplot
    axs[1].set_xlabel('Bus Stop Code')
    axs[1].set_ylabel('Average Tap-Out Volume')
    axs[1].set_title(f'Average Tap-Out Volume for Bus Service {service_no} by Stop Sequence')
    axs[1].set_xticks(range(len(passenger_demand_data[months[0]])))
    axs[1].set_xticklabels(passenger_demand_data[months[0]]['Description'], rotation=90, ha='right')
    axs[1].grid(axis='y')
    axs[1].legend()

    # Show the plots
    plt.tight_layout()
    plt.show()

In [ ]:
passenger_volume_65 = plot_monthly_passenger_demand(bus_routes_combined, passenger_volume_am, "65")
passenger_volume_67 = plot_monthly_passenger_demand(bus_routes_combined, passenger_volume_am, "67")
passenger_volume_170 = plot_monthly_passenger_demand(bus_routes_combined, passenger_volume_am, "170")

Obtaining route information using the OneMap API



In [ ]:
def get_coordinates(bus_stop_codes, bus_stops):

    chosen_stops = bus_stops[bus_stops['BusStopCode'].isin(bus_stop_codes)].copy()

    chosen_stops['BusStopCode'] = pd.Categorical(chosen_stops['BusStopCode'], categories=bus_stop_codes, ordered=True)

    chosen_stops = chosen_stops.sort_values('BusStopCode')

    chosen_stops['lat_lon'] = chosen_stops['Latitude'].astype(str) + ',' + chosen_stops['Longitude'].astype(str)

    return chosen_stops['lat_lon'].tolist()

Show all possible routes from a single start point to multiple end points using a specified mode of transport

In [ ]:
import requests

def get_route_info(start, end, MODE):
    params = {
        'start': start,
        'end': end,
        'mode': MODE,
        'routeType': 'pt',
        'date': '10-21-2024',
        'time': '08:30:00',
        'maxWalkDistance': 400,
        'numItineraries': 3,  # Get multiple itineraries
        'showIntermediateStops': 'true'  # Set this to true to get intermediate bus stops
    }

    headers = {'Authorization': onemap_api_key}
    url = "https://www.onemap.gov.sg/api/public/routingsvc/route"

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Function to display the exact route with total duration for each itinerary
def display_route_details(route_info, end_point):
    if 'plan' in route_info:
        print(f"Routes to End Point: {end_point}")
        for itinerary in route_info['plan']['itineraries']:
            print("\nNew Itinerary:")
            total_duration = 0  # Initialize total duration for the itinerary

            for leg in itinerary['legs']:
                leg_duration = leg['duration'] / 60  # Convert duration from seconds to minutes
                total_duration += leg_duration  # Add leg duration to the total

                print(f"Mode: {leg['mode']}")
                if leg['mode'].lower() in ['bus', 'subway']:
                    print(f"Service: {leg['route']}")
                    print(f"From Stop: {leg['from']['name']}, To Stop: {leg['to']['name']}")
                    print(f"Intermediate Stops: {[stop['name'] for stop in leg.get('intermediateStops', [])]}")

                print(f"Start Location: {leg['from']['name']} at {leg['from']['lat']}, {leg['from']['lon']}")
                print(f"End Location: {leg['to']['name']} at {leg['to']['lat']}, {leg['to']['lon']}")
                print(f"Duration: {leg_duration} minutes")
                print("-" * 30)

            # Print the total duration for the itinerary
            print(f"Total Duration of Itinerary: {total_duration:.2f} minutes")
            print("=" * 50)
    else:
        print("No route information found for endpoint:", end_point)


# Function to get routes for one start point with multiple end points
def get_routes_for_multiple_endpoints(start_point, end_points, mode):
    for end_point in end_points:
        route_info = get_route_info(start_point, end_point, mode)
        display_route_details(route_info, end_point)

Origin Destination Pairs for Bus 170

In [ ]:
origin_coordinates = get_coordinates([40011], bus_stops)
codes = [41021, 42051, 43061, 44021]
dest_coordinates = get_coordinates(codes, bus_stops)

mode = 'TRANSIT'
get_routes_for_multiple_endpoints(origin_coordinates, dest_coordinates, mode)

Origin Destination Pairs for Bus 65

In [ ]:
tamp_int_coords = get_coordinates([75009], bus_stops)
codes_65 = [71129, 70109, 60119, 13059, 14141]
dest_coordinates_65 = get_coordinates(codes_65, bus_stops)

mode = 'TRANSIT'
get_routes_for_multiple_endpoints(tamp_int_coords, dest_coordinates_65, mode)

In [ ]:
bencoolen_coords = get_coordinates([4029], bus_stops)
codes_65 = [13059, 14141]
dest_coordinates_65 = get_coordinates(codes_65, bus_stops)

mode = 'TRANSIT'
get_routes_for_multiple_endpoints(bencoolen_coords, dest_coordinates_65, mode)

Origin Destination Pairs for Bus 67

In [ ]:
cck_int_coords = get_coordinates([44009], bus_stops)
codes_67 = [44029, 42099, 40019]
dest_coordinates_67 = get_coordinates(codes_67, bus_stops)

mode = 'TRANSIT'
get_routes_for_multiple_endpoints(cck_int_coords, dest_coordinates_67, mode)